### Mount vào dir chứa model

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/ModelAI/SpamFilter'

Mounted at /content/gdrive
/content/gdrive/MyDrive/ModelAI/SpamFilter


### Cài các thư viện cần thiết

In [ ]:
!pip install scikit-learn underthesea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.2 MB/s eta 0:00:00


### Load dataset

In [ ]:
import os
DIR_ROOT = '/content/gdrive/MyDrive/ModelAI/SpamFilter'
DIR_DATASET = os.path.join(DIR_ROOT, 'dataset')


PATH_TRAIN = os.path.join(DIR_DATASET, 'train.csv')
PATH_DEV = os.path.join(DIR_DATASET, 'dev.csv')
PATH_TEST = os.path.join(DIR_DATASET, 'test.csv')


STOPWORDS_PATH = os.path.join(DIR_ROOT, 'vietnamese-stopwords.txt')


# Tạo dir model nếu chưa có
MODEL_DIR = os.path.join(DIR_ROOT, 'model')
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

### Load các từ trong tập train và tập test

In [ ]:
import pandas as pd


# Lấy các tập train và test
data_train = pd.read_csv(PATH_TRAIN)
data_test = pd.read_csv(PATH_TEST)


# Tạo tập train
x_train = data_train['Comment']
y_train = data_train['Label']
# Fill các trường null (nếu có) trong dataset
x_train = x_train.fillna('')    #Câu không có nội dung
y_train = y_train.fillna(1)     #Gán nhãn 1 nếu chưa có nhãn


# Tạo tập test
x_test = data_test['Comments']
y_test = data_test['Label']


# Tạo list stopword cho tiền xử lý
with open(STOPWORDS_PATH, 'r') as stopword_txt:
    list_stopwords = []
    for line in stopword_txt:
        #Đọc từng dòng trong txt và cho vào list
        word = line.strip('\n')
        list_stopwords.append(word)
    #Chuyển list stopword thành set
    list_stopwords = set(list_stopwords)

### Tiền xử lý dữ liệu với `underthesea`

In [ ]:
from underthesea import word_tokenize
import string, re


def filter_stop_word(tokenizers, list_stopwords):
    return [token for token in tokenizers if token not in list_stopwords]


# Lọc hết các emoji khỏi câu
def remove_emoji(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U0001F300-\U0001F5FF"  # Symbols & Pictographs
        u"\U0001F680-\U0001F6FF"  # Transport & Map symbols
        u"\U0001F1E0-\U0001F1FF"  # Flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)


def pre_processing_data(text):
    if type(text) is not str: return text
    
    #Chuyển về chữ thường
    text = text.lower()
    #Loại bỏ dấu câu
    text = text.translate(str.maketrans("", "", string.punctuation))
    #Tách từ thành list token
    tokenizers = word_tokenize(text)
    #Lọc stopword
    tokenizers = filter_stop_word(tokenizers, list_stopwords)
    #Tạo câu hoàn chỉnh dựa trên stopword
    sentences = " ".join(tokenizers)
    #Loại bỏ hết emoji
    sentences = remove_emoji(sentences)

    return sentences

### Tiến hành train

In [ ]:
#Import pipeline để chạy vertorize và sau đó xử lý bằng các thuật toán đã chọn
from sklearn.pipeline import Pipeline


#Vectorize hóa các từ trong văn bản bằng Bag of Word
from sklearn.feature_extraction.text import CountVectorizer


#Import thuật toán Multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB 
#RandomForest và Stacking để ghép 2 thuật toán lại
from sklearn.ensemble import RandomForestClassifier, StackingClassifier 
#Dùng để kiểm tra lại model
from sklearn.linear_model import LogisticRegression 


#Import các hàm để đánh giá
from sklearn.metrics import classification_report, accuracy_score


# Tiến hành tiền xử lý tập train
x_train = x_train.apply(pre_processing_data)


#Các classifier sử dụng:
rf = RandomForestClassifier(n_estimators=1000, random_state=1000)
nb = MultinomialNB(class_prior=[0.5, 0.5])


#Kết hợp lại (Dùng Logistic Regression làm meta)
stacking = StackingClassifier(estimators=[('nb', nb), ('rf', rf)], final_estimator=LogisticRegression())


# Tạo pipeline để quy định các bước train
pline = Pipeline([
    # Vectorize hóa input bằng Bag of Word
    ('vectorizer', CountVectorizer(ngram_range=(1, 10), max_features=85000)),
    # Tiến hành phân loại (classifier) bằng việc kết hợp 2 thuật toán
    ('classifier', stacking)
])


#Tiến hành train
pline.fit(x_train, y_train)


# Đánh giá trên tập test
y_pred_test = pline.predict(x_test)


# Hiển thị kết quả sau khi test
print("Độ chính xác ở tập test:", accuracy_score(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))

Độ chính xác ở tập test: 0.9441249864381035
              precision    recall  f1-score   support

           0       0.93      0.97      0.95      5231
           1       0.96      0.91      0.93      3986

    accuracy                           0.94      9217
   macro avg       0.95      0.94      0.94      9217
weighted avg       0.94      0.94      0.94      9217



### Đóng gói model lại

In [ ]:
import pickle
# Lưu model vào tệp .pkl
with open(os.path.join(MODEL_DIR, 'spamfilter_model.pkl'), 'wb') as f:
    pickle.dump(pline, f)

#### Cách sử dụng model

In [ ]:
# Tải lại model từ tệp .pkl
with open(os.path.join(MODEL_DIR, 'spamfilter_model.pkl'), 'rb') as f:
    loaded_pipeline = pickle.load(f)


# Sử dụng model đã tải để dự đoán
y_predict_loaded = loaded_pipeline.predict_proba(["Nhà này thật tuyệt vời, chủ nhà rất lịch sự. Chắc chắn sẽ ở lại đây lần nữa!!"])
print(y_predict_loaded)

[[0.98947216 0.01052784]]
